In [ ]:
!pip install torch torchvision torchaudio ultralytics pillow pytesseract numpy opencv-python matplotlib
!pip install scikit-learn matplotlib transformers 

In [1]:
check = [
    "check_signature_test_case/check/al_arafah_mehedy.jpeg",
    "check_signature_test_case/check/community_tareq.jpeg",
    "check_signature_test_case/check/ncc_anik.jpeg",
    "check_signature_test_case/check/bangla_check.jpeg",
]

In [2]:
import pytesseract
from PIL import Image, ImageFilter, ImageOps
import re
import json

pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"

def open_image_with_exif_correction(image_path):
    image = Image.open(image_path)
    image = ImageOps.exif_transpose(image)

    return image
    
def extract_check_no_from_corner(image):
    width, height = image.size
    # Define crop box (left, upper, right, lower)
    crop_box = (int(width * 0.60), 0, width, int(height * 0.40))  
    cropped = image.crop(crop_box)

    text = pytesseract.image_to_string(cropped, lang='eng')

    print("\n--- Check No. OCR Region Output ---\n")
    print(text)

    digit_sequences = re.findall(r'\d+', text)

    if not digit_sequences:
        return None
    
    # Pick the longest digit sequence as the check number
    check_no = max(digit_sequences, key=len)
    
    # Optional: Ensure 7 digits by zero-padding left if needed
    check_no = check_no.zfill(7)
    
    return check_no


def extract_check_info(image_path, lang='eng'):
    image = open_image_with_exif_correction(image_path)
    
    check_no_from_corner = extract_check_no_from_corner(image)

    # Convert to grayscale and reduce noise
    gray = ImageOps.grayscale(image)
    gray = gray.filter(ImageFilter.MedianFilter(size=3))

    # Extract text
    text = pytesseract.image_to_string(gray, lang=lang)
    print("\n--- OCR Output ---\n")
    print(text)

    lines = text.strip().splitlines()
    
    def extract_bank_name(text):
        match = re.search(r'\b([A-Za-z&]+\s+Bank(?:\s+(?:of\s+\w+|PLC|Ltd|Limited))?)\b', text, re.IGNORECASE)
        return match.group(1).strip() if match else None

    def extract_account_number(text):
        # Match full account numbers with optional hyphen (e.g., 0028-0310065175)
        match = re.search(r'\b\d{2,4}-\d{10,19}\b', text)
        if match:
            return match.group(0)
    
        # Fallback: just match plain 10–16 digit numbers
        match = re.search(r'\b\d{10,16}\b', text)
        return match.group(0) if match else None


    #def extract_check_no(text):
        #match = re.search(r'\b\d{6,7}\b', text)
        #if match:
            #return match.group(0).zfill(7)  # Ensure 7-digit format
        #return None

    def extract_date(text):
        match = re.search(r'\b(\d{2}[-/]\d{2}[-/]\d{4}|\d{8})\b', text)
        return match.group(1) if match else None

    def extract_account_name(lines):
        acc_number = extract_account_number(text)
        if not acc_number:
            return None

        for i, line in enumerate(lines):
            if acc_number in line:
                # Look at lines after account number
                for j in range(i + 1, len(lines)):
                    name_candidate = lines[j].strip()
                    if name_candidate:
                        # Remove numbers and symbols, keep only uppercase letters and spaces
                        cleaned = re.sub(r'[^A-Z\s]', '', name_candidate)
                        cleaned = re.sub(r'\s{2,}', ' ', cleaned).strip()
                        if cleaned and cleaned.isupper():
                            return cleaned
        return None


    extracted_data = {
        "bank_name": extract_bank_name(text),
        "account_number": extract_account_number(text),
        "check_no": check_no_from_corner,
        #"account_name": extract_account_name(lines),
        "date": extract_date(text)
    }

    print("\n--- Extracted Info ---\n")
    print(json.dumps(extracted_data, indent=2))

    return extracted_data

# Example usage
info = extract_check_info("check_signature_test_case/check/al_arafah_mehedy.jpeg")



--- Check No. OCR Region Output ---

“20 4229740

1s] D uh tA a ra y ¢


--- OCR Output ---

5° 4229740

ait lA) | rer re 29740
em ILO

| Svea ase fae

BANANI (015260435) Branch
ep
, Pay To a a a cecal Or Bearer
;
5 The Sum of Toke i‘ (sts
{ —_—_—________ Tk |
ee |
- eee
*  AL-WADIAH CURRENT ACCOUNT NO.
NAZTECH INC LIMITED Hi s
A/C No. 0201020015855 ee take
é Please Sign Above This Line

“G229?L0" O152604451 OcOsO200iS8s55 3



--- Extracted Info ---

{
  "bank_name": null,
  "account_number": "0201020015855",
  "check_no": "4229740",
  "date": null
}


In [ ]:
Database Interaction

In [25]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pymysql
import json  # Required to serialize the vector list
import gradio as gr

def execute_sql(sql_query: str, params=None):
    connection = None
    try:
        connection = pymysql.connect(
            host='10.33.32.61',
            user='root',
            password='Open@123',
            database='signature_verify',
            port=3306,
            cursorclass=pymysql.cursors.DictCursor
        )

        with connection.cursor() as cursor:
            cursor.execute(sql_query, params)

            # Fetch results if it's a SELECT query
            if sql_query.strip().lower().startswith("select"):
                result = cursor.fetchall()
                return result
            else:
                connection.commit()
                return "Query executed successfully."

    except Exception as e:
        return f"Error: {e}"

    finally:
        if connection:
            connection.close()

def insert_vector(account_number: str, signature_vector: list):
    # Convert the list to a JSON string
    vector_json = json.dumps(signature_vector)

    query = "INSERT INTO signatures (account_number, signature_vector) VALUES (%s, %s)"
    result = execute_sql(query, (account_number, vector_json.encode('utf-8')))  # encode for BLOB
    return result

def get_vector(account_number: str):
    query = "SELECT signature_vector FROM signatures WHERE account_number = %s"
    result = execute_sql(query, (account_number,))
    vectors = []
    if result:
        for row in result:
            blob = row['signature_vector']
            vector = json.loads(blob.decode('utf-8'))
            vectors.append(vector)
        return vectors
    return None

def fetch_account_name(account_number: str):
    query = "SELECT account_name FROM accounts WHERE account_number = %s"
    result = execute_sql(query, (account_number,))
    if result and isinstance(result, list) and len(result) > 0:
        return result[0]["account_name"]
    return "Account not found"

def fetch_account_numbers_from_t_bank_table(prefix: str):
    if not prefix.strip():
        return []  # Return empty list for empty search
    query = "SELECT account_number FROM t_bank_table WHERE account_number LIKE %s LIMIT 10"
    result = execute_sql(query, (f"{prefix}%",))
    return [row['account_number'] for row in result] if result else []


def fetch_account_numbers_by_prefix(prefix: str):
    if not prefix.strip():
        return []  # Return empty list for empty search
    query = "SELECT account_number FROM accounts WHERE account_number LIKE %s LIMIT 10"
    result = execute_sql(query, (f"{prefix}%",))
    return [row['account_number'] for row in result] if result else []

def select_account_number(account_numbers, evt: gr.SelectData):
    """Handle when user clicks on an account number from the search results"""
    selected_account = account_numbers[evt.index]
    return selected_account

def get_image_by_account_number(account_number):

    if not account_number.strip():
        return None, None
    
    query = "SELECT signature_1, signature_2 FROM t_bank_table WHERE account_number = %s"
    result = execute_sql(query, (account_number,))

    if not result:
        return None, None

    path1 = result[0]['signature_1']
    path2 = result[0]['signature_2']

    obj_1 = extract_object_name_from_url(path1)
    obj_2 = extract_object_name_from_url(path2)

    img_1 = get_image_from_minio(obj_1)
    img_2 = get_image_from_minio(obj_2)

    return img_1, img_2

def fetch_bulk_signature_info(limit=10):
    query = "SELECT account_number, account_name, flag, signature_1, signature_2, comments, status FROM t_bank_table ORDER BY created_at DESC LIMIT %s"
    result = execute_sql(query, (limit,))
    return result if result else []

def get_all_account_number():
    query = "SELECT account_number FROM t_bank_table"
    result = execute_sql(query)
    return [row['account_number'] for row in result] if result else[]


# Test
account_number = "0201020015855"  # Must be int for BIGINT
vector_list = [0.1, 0.2, 0.3]

#print(insert_vector(account_number, vector_list))
print(get_vector(account_number))


/Users/shawon/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[-0.00850657094269991, -0.04479064792394638, -0.035481005907058716, -0.001488094567321241, 0.004947292152792215, -0.009149120189249516, 0.036534570157527924, -0.0411672480404377, 0.060147833079099655, 0.012520482763648033, 0.01704329438507557, -0.010122961364686489, 0.0016728986520320177, 0.004304077010601759, -0.00694573437795043, 0.02378014661371708, 0.03520826995372772, 0.048639316111803055, 0.04429136961698532, 0.017158687114715576, 0.03166569024324417, 0.03410753607749939, 0.0019485305529087782, -0.038373496383428574, -0.002612387528643012, -0.03200182318687439, 0.043060336261987686, 0.060374557971954346, -0.0033807384315878153, 0.005803059786558151, -0.054184168577194214, -0.08967618644237518, -0.0073558371514081955, -0.001638060319237411, -0.03176342323422432, -0.03869050741195679, -0.015207641758024693, -0.019591843709349632, 0.02604702115058899, 0.04763595387339592, -0.002764028264209628, -0.019710825756192207, -0.01749362237751484, -0.042385417968034744, -0.00059139664517715

In [ ]:
Get Imgae form minIO

In [ ]:
!pip install minio

In [4]:
from minio import Minio
from io import BytesIO
from PIL import Image

def get_image_from_minio(object_path, bucket_name = "signature"):
    client = Minio(
        endpoint="10.33.11.1:9000",
        access_key="minioadmin",
        secret_key="minioadmin",
        secure=False
    )

    try:
        response = client.get_object(bucket_name, object_path)

        image = Image.open(BytesIO(response.read()))
        return image

    except Exception as e:
        print(f"Error fetching image from MinIO: {e}")
        return None

In [6]:
MinIO expects only the object name, not the whole url

SyntaxError: invalid syntax (1259978411.py, line 1)

In [5]:
from urllib.parse import urlparse

def extract_object_name_from_url(url):
    parsed = urlparse(url)
    path = parsed.path.lstrip('/') 
    
    # Remove the first part if it matches the bucket name
    parts = path.split('/')
    if parts[0] == "signature":
        parts = parts[1:] 
    return '/'.join(parts)



In [8]:
get_image_by_account_number("00119604443556")

Error fetching image from MinIO: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /signature/shwaon_1.jpeg, request_id: 184DCC1F3609179B, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: signature, object_name: shwaon_1.jpeg


(None, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x839>)

In [9]:
YOLOv8s detection and Image processing - binary image

SyntaxError: invalid syntax (178537443.py, line 1)

In [6]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

def isolate_signature(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold to binary image (signature in white, background in black)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create empty mask to draw filtered contours
    mask = np.zeros_like(binary)

    # Filter and draw only meaningful signature-like contours
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)

        # Filter conditions
        if area > 100 and w > 10 and h > 10:  # Skip tiny specks
            mask = cv2.drawContours(mask, [cnt], -1, 255, -1)  # Fill the contour

    # Final result (bitwise AND to extract the denoised signature)
    result = cv2.bitwise_and(binary, mask)

    return result

# Load model and image
def process_signature_image(image_path):
    model = YOLO("yolov8s.pt")
    image = cv2.imread(image_path)
    results = model(image)

    if results[0].boxes is None or len(results[0].boxes) == 0:
        print("No detection from yolov8s")
        return None

    processed_images = []
    for i, box in enumerate(results[0].boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cropped = image[y1:y2, x1:x2]
        cleaned = isolate_signature_with_line_removal(cropped)
        processed_images.append(cleaned)

    return processed_images  # List of processed images


In [ ]:
isolate_signature with line removal

In [47]:
import os
import cv2
import numpy as np

def isolate_signature_with_line_removal(image):
    """Isolate signature, remove a single horizontal line near the bottom and its nearby noise only if the line exists."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold to binary (white signature on black background)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    height = binary.shape[0]
    width = binary.shape[1]
    
    # Focus only on the lower part where the horizontal line is expected
    lower_part = binary[int(height * 0.7):]  # bottom 30% of the image
    
    # Use a much longer horizontal kernel to detect actual lines (not signature strokes)
    # The kernel should be at least 20-30% of image width to ensure it's a real line
    min_line_length = max(80, int(width * 0.50))  # At least 80px or 25% of width
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (min_line_length, 1))
    detected_line = cv2.morphologyEx(lower_part, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
    
    # Additional validation: check if detected line is actually a continuous horizontal line
    line_exists = False
    if cv2.countNonZero(detected_line) > 0:
        # Find contours in the detected line
        line_contours, _ = cv2.findContours(detected_line, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in line_contours:
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if this is actually a horizontal line:
            # 1. Width should be significant (at least 20% of image width)
            # 2. Height should be small (line-like, not a blob)
            # 3. Aspect ratio should be very wide
            aspect_ratio = w / h if h > 0 else 0
            
            if (w >= int(width * 0.2) and  # At least 20% of image width
                h <= 5 and                # Very thin (line-like)
                aspect_ratio >= 15):      # Very wide aspect ratio
                line_exists = True
                break
    
    # Only remove line if it actually exists and meets our criteria
    if line_exists:
        # Map the line back to original coordinates
        line_mask = np.zeros_like(binary)
        line_mask[int(height * 0.7):] = detected_line
        
        # Subtract the detected line
        binary = cv2.subtract(binary, line_mask)
        
        # Get the y-coordinate of the horizontal line (relative to full image)
        line_y_positions = np.where(line_mask > 0)[0]
        if len(line_y_positions) > 0:
            y_line = np.min(line_y_positions)
            y_start = max(0, y_line - 2)  # Small buffer above the line
            y_end = min(y_line + 8, height)  # Small buffer below the line
            binary[y_start:y_end, :] = 0  # remove noise around the line
    
    # Find contours and keep only large, valid ones
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(binary)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        # Filter conditions for meaningful signature parts
        if area > 100 and w > 10 and h > 10:
            cv2.drawContours(mask, [cnt], -1, 255, -1)
    
    result = cv2.bitwise_and(binary, mask)
    return result.astype(np.uint8)

In [ ]:
Extract Vector by ViT and calculate cosine similarity

In [8]:
from transformers import AutoImageProcessor, DeiTModel
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load DeiT model and processor
processor = AutoImageProcessor.from_pretrained("facebook/deit-base-distilled-patch16-224")
model = DeiTModel.from_pretrained("facebook/deit-base-distilled-patch16-224", local_files_only=True)

def extract_vector(image_array):
    image = Image.fromarray(cv2.cvtColor(image_array, cv2.COLOR_GRAY2RGB))
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token embedding
    vector = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    vector /= np.linalg.norm(vector)
    return vector

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def get_similarity(check_signature, stored_signature):
    vec1 = extract_vector(check_signature)
    vec2 = extract_vector(stored_signature)
    similarity = cosine_similarity(vec1, vec2)
    return similarity

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of DeiTModel were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
Two sample image to process image plot 

In [9]:
import matplotlib.pyplot as plt

def get_process_signature(check_path, signature_path):
    check_images = process_signature_image(check_path)
    stored_images = process_signature_image(signature_path)

    if not check_images or not stored_images:
        print("No signatures detected.")
        return

    check_signature = check_images[0]
    stored_signature = stored_images[0]

    similarity = get_similarity(check_signature, stored_signature)
    print("Cosine Similarity:", similarity)

    plot_signature(check_signature, stored_signature)

    if similarity >= 0.80:
        print("Genuine Signature")
    else:
        print("Forged Signature")


def plot_signature(check_signature, stored_signature):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(check_signature, cmap='gray')
    axs[0].set_title("Uploaded Signature")
    axs[0].axis('off')

    axs[1].imshow(stored_signature, cmap='gray')
    axs[1].set_title("Reference Signature")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()


In [10]:
def run_comparisons(index):
    if index < 1 or index > len(check):
        print("Invalid index: Please use a number between 1 and", len(check))
        return

    check_path = check[index - 1]
    print(f"Running comparisons for: {check_path}\n")

    for sig_path in signatures:
        print(f"Comparing with: {sig_path}")
        (get_process_signature(check_path, sig_path))
        print("-" * 140)


In [15]:
Send two image for signature detection, image processing, vector embedding and cosine similarity

SyntaxError: invalid syntax (1675066724.py, line 1)

In [11]:
def get_comparison(check_path, signature_path):
    check_images = process_signature_image(check_path)
    signature_images = process_signature_image(signature_path)

    if not check_images or not signature_images:
        print("No signatures detected.")
        return

    check_sig = check_images[0]
    sig_sig = signature_images[0]

    similarity = get_similarity(check_sig, sig_sig)
    print("Cosine Similarity:", similarity)

    plot_signature(check_sig, sig_sig)

    return similarity

In [ ]:
Insert into database

In [12]:
def insert_vector_db(account_number, signature_path):
    process_image = process_signature_image(signature_path)

    #if process_image is None:
        #print("No Detection By Model")
        #return "No Detection By Model"
        
    first_signature = process_image[0]
    
    signature_vector = extract_vector(first_signature).tolist()

    result = insert_vector(account_number, signature_vector)

    return result
    

In [32]:
insert_vector_db("0028", "check_signature_test_case/signature/bangla_1.jpeg")


0: 576x640 1 signature, 72.5ms
Speed: 2.1ms preprocess, 72.5ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 640)


'Query executed successfully.'

In [ ]:
Process check signature with database existing vector

In [13]:
def get_similarity_from_db(account_number: int, check_signature_image_path: str):
    stored_vectors = get_vector(account_number)
    if stored_vectors is None:
        print("No vector found in DB.")
        return None, "Account Not Authorized"

    # Extract vector from new image
    check_image_array = process_signature_image(check_signature_image_path)

    if not check_image_array:
        print("No signatures detected.")
        return None, "Not Found"

    first_signature = check_image_array[0]
    
    check_vector = extract_vector(first_signature)

    similarities = [
        cosine_similarity(check_vector, np.array(stored))
        for stored in stored_vectors
    ]

    return float(max(similarities)), None


In [10]:
get_similarity_from_db("0028-0310065175", "check_signature_test_case/signature/anik_2.jpeg")


0: 256x640 1 signature, 47.5ms
Speed: 1.7ms preprocess, 47.5ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 640)


(0.9999999951625704, None)

In [11]:
result = extract_check_info("check_signature_test_case/check/al_arafah_mehedy.jpeg")
account_number = result['account_number']
bank_name = result['bank_name']

get_similarity_from_db(account_number, "check_signature_test_case/check/al_arafah_mehedy.jpeg")


--- Check No. OCR Region Output ---

“20 4229740

1s] D uh tA a ra y ¢


--- OCR Output ---

5° 4229740

ait lA) | rer re 29740
em ILO

| Svea ase fae

BANANI (015260435) Branch
ep
, Pay To a a a cecal Or Bearer
;
5 The Sum of Toke i‘ (sts
{ —_—_—________ Tk |
ee |
- eee
*  AL-WADIAH CURRENT ACCOUNT NO.
NAZTECH INC LIMITED Hi s
A/C No. 0201020015855 ee take
é Please Sign Above This Line

“G229?L0" O152604451 OcOsO200iS8s55 3



--- Extracted Info ---

{
  "bank_name": null,
  "account_number": "0201020015855",
  "check_no": "4229740",
  "date": null
}

0: 320x640 1 signature, 53.5ms
Speed: 1.3ms preprocess, 53.5ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)


(0.8589157413425498, None)

In [36]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
Tab 4: Bulk Generator

In [14]:
def save_file_tab3(uploaded_file, save_dir="uploaded_image"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    timestamp = str(int(time.time() * 1000))  # milliseconds
    filename = f"uploaded_{timestamp}.jpeg"
    save_path = os.path.join(save_dir, filename)

    if uploaded_file.mode == 'RGBA':
        uploaded_file = uploaded_file.convert("RGB")
    
    uploaded_file.save(save_path)
    return save_path

def bulk_vector_generator():
    try:
        account_numbers = get_all_account_number()
        if not account_numbers:
            return "No account numbers found."

        for account_number in account_numbers:
            img1, img2 = get_image_by_account_number(account_number)

            for i, img in enumerate([img1, img2], start=1):
                try:
                    if img is not None:
                        img_path = save_file_tab3(img)
                        insert_vector_db(account_number, img_path)
                except Exception as inner_e:
                    return f"Error processing signature {i} for account {account_number}: {inner_e}"
                    
        return "Bulk Vector generation Completed Sucessfully"
        
    except Exception as e:
        return f"Error in bulk_vector_generator: {e}"

                

In [77]:
Tab2: Signature Verification from check

SyntaxError: invalid syntax (1515166345.py, line 1)

In [15]:
import time
import os

def save_uploaded_file_tab2(uploaded_file, save_dir="uploaded_image"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    timestamp = str(int(time.time() * 1000))  # milliseconds
    filename = f"uploaded_{timestamp}.jpeg"
    save_path = os.path.join(save_dir, filename)

    if uploaded_file.mode == 'RGBA':
        uploaded_file = uploaded_file.convert("RGB")
    
    uploaded_file.save(save_path)
    return save_path


def process_check_image_tab2(image):
    image_path = save_uploaded_file(image)

    # Step 1: OCR to extract info
    result = extract_check_info(image_path)
    print(image_path)
    print(result)
    
    account_number = result.get('account_number', '')
    bank_name = result.get('bank_name', '')
    check_no = result.get('check_no', '')

    img_1, img_2 = get_image_by_account_number(account_number)

    img1_path = None
    simr_1 = "N/A"
    if img_1 is not None:
        img1_path = save_uploaded_file_tab2(img_1)
        simr_1 = get_comparison(image_path, img1_path)

    img2_path = None
    simr_2 = "N/A"
    if img_2 is not None:
        img2_path = save_uploaded_file_tab2(img_2)
        simr_2 = get_comparison(image_path, img2_path)


    # Step 2: Sanitize account number (remove dashes etc.)
    #account_number = re.sub(r'\D', '', account_number_raw)

    # Step 3: Run similarity
    similarity, error_message = get_similarity_from_db(account_number, image_path)

    if error_message:
        similarity_str = error_message
        status_str = "Unknown"

    else:
        similarity_str = (
            round(similarity, 4) if similarity is not None else "No Match Found"
        )
        status_str = "Genuine" if similarity_str >= 0.80 else "Forged"

    return img_1, simr_1, img_2, simr_2, bank_name, account_number, check_no, similarity_str, status_str


In [121]:
import gradio as gr
import re

# Create Gradio Interface
interface = gr.Interface(
    fn=process_check_image,
    inputs=gr.Image(type="pil", label="Upload Check Image"),
    outputs=[
        gr.Textbox(label="Bank Name"),
        gr.Textbox(label="Account Number"),
        gr.Textbox(label="Check No."),
        gr.Textbox(label="Similarity Score"),
        gr.Textbox(label="Signature Status")
    ],
    title="Signature Verification from Check",
    description="Upload a scanned check. The system will extract bank and account info, then check signature similarity."
)

interface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [32]:
get_process_signature("check_signature_test_case/signature/tareq_1.jpeg" ,"check_signature_test_case/signature/tareq_2.jpeg")


0: 640x576 1 signature, 175.2ms
Speed: 4.9ms preprocess, 175.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 576)

0: 416x640 1 signature, 90.3ms
Speed: 2.1ms preprocess, 90.3ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)
Cosine Similarity: 0.8968249


<Figure size 1000x400 with 2 Axes>

Genuine Signature


In [ ]:
Tab1: Insert vector into vector database

In [16]:
import gradio as gr
import time
import os

def save_uploaded_file_tab1(uploaded_file, save_dir="uploaded_signatures"):
    os.makedirs(save_dir, exist_ok=True)
    timestamp = int(time.time())
    filename = f"signature_{timestamp}.jpeg"
    save_path = os.path.join(save_dir, filename)
    uploaded_file.save(save_path)
    return save_path

def insert_signature(account_number: str, signature_image):
    # Decide which account number to use:
    # Use dropdown selection if it exists and is not empty
    #account_number = selected_account_number if selected_account_number else typed_account_number
    
    if not account_number:
        return "Account number is required."
    if signature_image is None:
        return "Signature image is required."

    image_path = save_uploaded_file_tab1(signature_image)
    try:
        result = insert_vector_db(account_number, image_path)
        return "Signature vector inserted successfully." if result else "Insertion failed."
    except Exception as e:
        return f"Error: {str(e)}"


In [122]:
upload_interface = gr.Interface(
    fn=insert_signature,
    inputs=[
        gr.Textbox(label="Account Number"),
        gr.Image(type="pil", label="Upload Your Signature Image"),
    ],
    outputs=gr.Textbox(label="Result"),
    title="Insert Signature Vector into DB",
    description="Upload your signature and link it to an account number to store its vector in the database."
)

upload_interface.launch(debug=True, server_port=7861)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
Tab3: Run Batch Check Verifier

In [37]:
import gradio as gr
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

check_paths = [
    "check_signature_test_case/check/jahidul_check.jpeg",
    "check_signature_test_case/check/shawon_check.jpg",
    "check_signature_test_case/check/al_arafah_mehedy.jpeg",
    "check_signature_test_case/check/community_tareq.jpeg",
    "check_signature_test_case/check/meherun_check.jpeg",
    "check_signature_test_case/check/nasren_check.jpeg",
    "check_signature_test_case/check/asif_check.jpeg",
    "check_signature_test_case/check/neshat_check.jpeg",
    "check_signature_test_case/check/mr_check.jpeg",
]

# This function processes a single check
def process_single_check(path, threshold):
    try:
        result = extract_check_info(path)
        account_number = result.get('account_number') or "A/C No. Extraction Failed"
        bank_name = result.get('bank_name') or "Not Recognized"
        check_no = result.get('check_no') or "Unreadable"

        similarity, error_message = get_similarity_from_db(account_number, path)

        if error_message:
            similarity_str = error_message
            status = "Fail"
        else:
            similarity_percentage = round(similarity * 100, 0)
            similarity_str = f"{similarity_percentage}%"
            status = "Pass" if similarity_percentage >= threshold else "Fail"

        return {
            "Check No.": check_no,
            "Bank": bank_name,
            "Account": account_number,
            "Similarity": similarity_str,
            "Status": status
        }

    except Exception as e:
        return {
            "Check No.": "Error",
            "Bank": "Error",
            "Account": "Error",
            "Similarity": "Error",
            "Status": f"Failed: {str(e)}"
        }

# Batch function with parallel processing
def run_batch_checks(threshold):
    results = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(process_single_check, path, threshold): path for path in check_paths}
        for future in as_completed(futures):
            results.append(future.result())

    df = pd.DataFrame(results)
    return df


In [ ]:

# Gradio Interface
load_all_check = gr.Interface(
    fn=run_batch_checks,
    inputs=[],
    outputs=gr.Dataframe(headers=["Check No.", "Bank", "Account", "Similarity", "Status"]),
    title="Batch Check Signature Verifier",
    description="Click the button to process all preloaded checks and get results with similarity score and Pass/Fail."
)

load_all_check.launch(debug=True, server_port=7862)

In [18]:
import gradio as gr
import pandas as pd

check_paths = [
    "check_signature_test_case/check/jahidul_check.jpeg",
    "check_signature_test_case/check/shawon_check.jpg",
    "check_signature_test_case/check/al_arafah_mehedy.jpeg",
    "check_signature_test_case/check/community_tareq.jpeg",
    "check_signature_test_case/check/meherun_check.jpeg",
    "check_signature_test_case/check/nasren_check.jpeg",
    "check_signature_test_case/check/asif_check.jpeg",
    "check_signature_test_case/check/neshat_check.jpeg",
    "check_signature_test_case/check/mr_check.jpeg",
]

def run_batch_check():
    results = []
    for path in check_paths:
        try:
            result = extract_check_info(path)
            account_number = result.get('account_number') or "No Account Number Detected"
            bank_name = result.get('bank_name') or "Not Recognized"
            check_no = result.get('check_no') or "Unreadable"
            
            similarity, error_message = get_similarity_from_db(account_number, path)

            if error_message:
                similarity_str = error_message
                status = "Fail"
            else:
                similarity_percentage = round(similarity * 100, 0)
                similarity_str = f"{similarity_percentage}%"
                status = "Pass" if similarity_percentage >= 80 else "Fail"

            results.append({
                "Check No.": check_no,
                "Bank": bank_name,
                "Account": account_number,
                "Similarity": similarity_str,
                "Status": status
            })

        except Exception as e:
            results.append({
                "Check No.": "Error",
                "Bank": "Error",
                "Account": "Error",
                "Similarity": "Error",
                "Status": f"Failed: {str(e)}"
            })

    df = pd.DataFrame(results)
    return df

# Gradio Interface
load_all_check = gr.Interface(
    fn=run_batch_check,
    inputs=[],
    outputs=gr.Dataframe(headers=["Check No.", "Bank", "Account", "Similarity", "Status"]),
    title="Batch Check Signature Verifier",
    description="Click the button to process all preloaded checks and get results with similarity score and Pass/Fail."
)

load_all_check.launch(debug=True, server_port=7862)


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
Tab5: Compare Two Signature

In [35]:
#For tab 4
import time
import os

def get_signature_similarity(check_path, signature_path):
    check_images = process_signature_image(check_path)
    signature_image = process_signature_image(signature_path)

    if not check_images or not signature_image:
        return None, "No signatures detected."

    check_signature = check_images[0]
    other_signature = signature_image[0]

    similarity = get_similarity(check_signature, other_signature)

    if similarity is None:
        return None, "Similarity Calculation Error"
    else:
        return similarity, None

def save_uploaded_file(uploaded_file, save_dir="uploaded_image"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    timestamp = str(int(time.time() * 1000))  # milliseconds
    filename = f"uploaded_{timestamp}.jpeg"
    save_path = os.path.join(save_dir, filename)
    uploaded_file.save(save_path)
    return save_path


def process_two_signature(first_image, second_image):
    image_path_1 = save_uploaded_file(first_image)
    image_path_2 = save_uploaded_file(second_image)

    similarity, error_message = get_signature_similarity(image_path_1, image_path_2)

    #delete images
    try:
        os.remove(image_path_1)
        os.remove(image_path_2)
    except Exception as e:
        print(f"Cleanup failed: {e}")

    if error_message:
        similarity_str = error_message
        status_str = "Unknown"
    else:
        similarity_str = round(similarity, 4) if similarity is not None else "No Match Found"
        status_str = "Genuine" if similarity >= 0.80 else "Forged"

    return similarity_str, status_str


    

1750680377.144395


In [ ]:
Main Gradio UI

In [54]:
import gradio as gr
import pandas as pd


def tab1_ui():
    with gr.Tab("1. Add Signature Vector"):
        gr.Markdown("## Upload your signature and link it to an account number to store its vector in the database.")
        with gr.Row():
            account_input = gr.Textbox(label="Type Account Number", placeholder="Start typing account number...")
            account_name = gr.Textbox(label="Account Name", interactive=False)

        search_results = gr.Radio(choices=[], label="Search Results", visible=False, interactive=True)

        with gr.Row():
            signature_image = gr.Image(type="pil", label="Upload Your Signature Image", height=300, width=300)
            insert_button = gr.Button("Add Signature")

        result_output = gr.Textbox(label="Result", interactive=False)

        def handle_input_change(prefix):
            if not prefix or not prefix.strip():
                return gr.update(choices=[], visible=False), ""

            accounts = fetch_account_numbers_by_prefix(prefix)
            name = fetch_account_name(prefix) 

            if accounts:
                return gr.update(choices=accounts, value=None, visible=True), name
            else:
                return gr.update(choices=[], visible=False), name

        def handle_radio_selection(selected_account):
            if not selected_account:
                return gr.update(), "", gr.update(visible=False)

            name = fetch_account_name(selected_account)
            return selected_account, name, gr.update(visible=False, choices=[])

        account_input.change(
            fn=handle_input_change,
            inputs=[account_input],
            outputs=[search_results, account_name]
        )

        search_results.input(
            fn=handle_radio_selection,
            inputs=[search_results],
            outputs=[account_input, account_name, search_results]
        )

        insert_button.click(
            fn=insert_signature,
            inputs=[account_input, signature_image],
            outputs=result_output
        )

        
def tab2_ui():
    with gr.Tab("2. Signature Verification from Check"):
        gr.Markdown("## Upload a scanned check. The system will extract bank and account info, then check signature similarity.")
        with gr.Row():
            check_image = gr.Image(type="pil", label="Upload Check Image", height=300, width=300)
            #signature_check = gr.Image(type="pil", label="Extracted Signature from Check", interactive=False)

        verify_button = gr.Button("Verify Check")

        with gr.Row():
            signature_1 = gr.Image(type="pil", label="Reference Signature 1", interactive=False, height=300, width=300)
            signature_2 = gr.Image(type="pil", label="Reference Signature 2", interactive=False, height=300, width=300)
            
        with gr.Row():
            signature_1_similarity = gr.Textbox(label="Signature 1 Similarity")
            signature_2_similarity = gr.Textbox(label="Signature 2 Similarity")

        with gr.Row():
            bank_name = gr.Textbox(label="Bank Name")
            account_number = gr.Textbox(label="Account Number")
            check_no = gr.Textbox(label="Check No.")
            
        with gr.Row():
            similarity_score = gr.Textbox(label="Similarity Score")
            signature_status = gr.Textbox(label="Signature Status")
            
        verify_button.click(
            fn=process_check_image_tab2,
            inputs=check_image,
            outputs=[signature_1, signature_1_similarity, signature_2, signature_2_similarity, bank_name, account_number, check_no, similarity_score, signature_status]
        )

def tab3_ui():
    with gr.Tab("3. Batch Check Signature Verifier"):
        gr.Markdown("## Click the button to process all preloaded checks and get results with similarity score and Pass/Fail.")

        batch_result = gr.Dataframe(headers=["Check No.", "Bank", "Account", "Similarity", "Status"], interactive=False)

        with gr.Row():
            threshold_slider = gr.Slider(label="Pass Threshold (%)", minimum=0, maximum=100, value=80, step=1)
            run_button = gr.Button("Run Batch Checks")

        account_input = gr.Textbox(label="Type Account Number", placeholder="Start typing account number...")
        search_results = gr.Radio(choices=[], label="Search Results", visible=False, interactive=True)

        with gr.Row():
            signature_1 = gr.Image(type="pil", label="Reference Signature 1", interactive=False, height=300, width=300)
            signature_2 = gr.Image(type="pil", label="Reference Signature 2", interactive=False, height=300, width=300)

        def handle_input_change_tab3(prefix):
            if not prefix or not prefix.strip():
                return gr.update(choices=[], visible=False)

            accounts = fetch_account_numbers_from_t_bank_table(prefix)

            if accounts:
                return gr.update(choices=accounts, value=None, visible=True)
            else:
                return gr.update(choices=[], visible=False)

        def handle_radio_selection_tab3(selected_account):
            if not selected_account:
                return gr.update(), "", gr.update(visible=False)

            img1, img2 = get_image_by_account_number(selected_account)
            return img1, img2, gr.update(visible=False, choices=[])

        account_input.change(
            fn=handle_input_change_tab3,
            inputs=[account_input],
            outputs=[search_results]
        )

        search_results.input(
            fn=handle_radio_selection_tab3,
            inputs=[search_results],
            outputs=[signature_1, signature_2]
        )

        run_button.click(
            fn=run_batch_checks,
            inputs=[threshold_slider],
            outputs=[batch_result]
        )

def tab4_ui():
    with gr.Tab("4. Bulk Signature Upload"):
        gr.Markdown("## Upload the Bulk Signature to store as a Vector")

        fetch_btn = gr.Button("Fetch Sample Records")
        df_output = gr.Dataframe(headers=["Account Number", "Account Name", "Flag", "Signature 1", "Signature 2", "Comments", "Status"], interactive=False, wrap=False)

        bulk_vector_btn = gr.Button("Run Bulk Vector Generator")
        bulk_result = gr.Textbox(label="Result", interactive=False)

        def load_bulk_data():
            rows = fetch_bulk_signature_info()
            if not rows:
                return []
            return [[
                row["account_number"],
                row["account_name"],
                row["flag"],
                row["signature_1"],
                row["signature_2"],
                row["comments"],
                row["status"]
            ] for row in rows]

        def call_bulk_generator():
            result = bulk_vector_generator()
            return result

        fetch_btn.click(fn=load_bulk_data, inputs=[], outputs=[df_output])
        bulk_vector_btn.click(fn=call_bulk_generator, inputs=[], outputs=[bulk_result])

        
        
def tab5_ui():
    with gr.Tab("5. Compare Two Signature"):
        gr.Markdown("## Upload two signature to get a quick similarity")
        with gr.Row():
            check_image_1 = gr.Image(type="pil", label="Upload Signature Image")
            signature_image_2 = gr.Image(type="pil", label="Upload Signature Image")
        with gr.Row():
            similarity_str = gr.Textbox(label="Similarity Score")
            status_str = gr.Textbox(label="Signature Status")
        verify_button = gr.Button("Verify Check")
        verify_button.click(
            fn= process_two_signature,
            inputs=[check_image_1, signature_image_2],
            outputs= [similarity_str, status_str]
        )

def tab6_ui():
    with gr.Tab("5. Compare Two Signature"):
        gr.Markdown("## Signature Image Processsing, Graph (GATs) and Vision Transformer (ViT)")

        with gr.Row():
            signature1 = gr.Image(type="pil", label="Upload Signature 1 Image", height=300, width=300)
            signature2 = gr.Image(type="pil", label="Upload Signature 2 Image", height=300, width=300)

        with gr.Row():
            process_button1 = gr.Button("Process Signature 1")
            process_button2 = gr.Button("Process Signature 2")

        with gr.Row():
            binary_signature1 = gr.Image(type="pil", label="Binary Image 1", interactive=False, height=300, width=300)
            binary_signature2 = gr.Image(type="pil", label="Binary Image 2", interactive=False, height=300, width=300)

        with gr.Row():
            skeleton_signature1 = gr.Image(type="pil", label="Skeleton 1", interactive=False, height=300, width=300)
            skeleton_signature2 = gr.Image(type="pil", label="Skeleton 2", interactive=False, height=300, width=300)
            
        with gr.Row():
            node_signature1 = gr.Image(type="pil", label="Node Detection 1", interactive=False, height=300, width=300)
            node_signature2 = gr.Image(type="pil", label="Node Detection 2", interactive=False, height=300, width=300)

        with gr.Row():
            graph_structure1 = gr.Image(type="pil", label="Graph Structure 1", interactive=False, height=300, width=300)
            graph_structure2 = gr.Image(type="pil", label="Graph Structure 2", interactive=False, height=300, width=300)
        
        with gr.Row():
            clean_graph1 = gr.Image(type="pil", label="Clean Graph Image 1", interactive=False, height=300, width=300)
            clean_graph2 = gr.Image(type="pil", label="Clean Graph Image 2", interactive=False, height=300, width=300)

        with gr.Row():
            enhanced_graph1 = gr.Image(type="pil", label="Enhanced Graph Image 1", interactive=False, height=300, width=300)
            enhanced_graph2 = gr.Image(type="pil", label="Enhanced Graph Image 2", interactive=False, height=300, width=300)

        with gr.Row():
            vector1=gr.Textbox(label="Vector Embeddings for Signature 1")
            vector2=gr.Textbox(label="Vector Embeddings for Signature 2")

        similarity_button = gr.Button("Similarity")

        with gr.Row():
            similarity=gr.Textbox(label="Cosine Similarity Calculation")

        process_button1.click(
            fn= image_processing_tab6,
            inputs=[signature1],
            outputs=[binary_signature1, skeleton_signature1, node_signature1, graph_structure1, clean_graph1, enhanced_graph1, vector1]
        )

        process_button2.click(
            fn= image_processing_tab6,
            inputs=[signature2],
            outputs=[binary_signature2, skeleton_signature2, node_signature2, graph_structure2, clean_graph2, enhanced_graph2, vector2]
        )

        similarity_button.click(
            fn= cosine_similarity_tab6,
            inputs=[],
            outputs=[similarity]
        )

with gr.Blocks(title="Signature Check System") as demo:
    gr.Markdown("# Bank Check Signature Verification")

    with gr.Tabs():
        tab1_ui()
        tab2_ui()
        tab3_ui()
        tab4_ui()
        #tab5_ui()
        tab6_ui()

#demo.launch(server_port=7860, debug=True)
demo.launch(server_name="0.0.0.0", server_port=7860, debug=True)


* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.



0: 384x640 1 signature, 50.3ms
Speed: 1.4ms preprocess, 50.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 512x640 1 signature, 66.9ms
Speed: 1.5ms preprocess, 66.9ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
Keyboard interruption in main thread... closing server.


In [ ]:
Tab 06: Image Processing

In [53]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx
from collections import deque
from skimage.morphology import skeletonize
from scipy.ndimage import convolve
from ultralytics import YOLO
from PIL import Image
import ast


def skeletonize_image(binary_signature):
    """Skeletonize the signature image"""
    # Make sure it's in [0, 1]
    binary = (binary_signature > 0).astype(np.uint8)
    skeleton = skeletonize(binary)
    return skeleton.astype(np.uint8)

def detect_nodes(skeleton):
    """Extract Nodes (Endpoints and Junctions)"""
    kernel = np.array([[1,1,1],
                       [1,10,1],
                       [1,1,1]])
    
    convolved = convolve(skeleton, kernel, mode='constant', cval=0)
    
    nodes = np.zeros_like(skeleton)
    
    for y in range(1, skeleton.shape[0] - 1):
        for x in range(1, skeleton.shape[1] - 1):
            if skeleton[y, x] == 1:
                value = convolved[y, x] - 10
                if value == 1:  # endpoint
                    nodes[y, x] = 1
                elif value > 2:  # junction
                    nodes[y, x] = 1

    return nodes

def get_neighbors(y, x, skeleton):
    """Get all valid skeleton neighbors of a pixel"""
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                  (-1, -1), (-1, 1), (1, -1), (1, 1)]
    neighbors = []
    h, w = skeleton.shape
    
    for dy, dx in directions:
        ny, nx = y + dy, x + dx
        if 0 <= ny < h and 0 <= nx < w and skeleton[ny, nx] == 1:
            neighbors.append((ny, nx))
    
    return neighbors

def trace_path_between_nodes(start_node, direction, skeleton, nodes, visited_global):
    """
    Trace a path from start_node in a given direction until reaching any node.
    """
    sy, sx = start_node
    dy, dx = direction
    current_y, current_x = sy + dy, sx + dx
    h, w = skeleton.shape
    
    # Check bounds and if it's a skeleton pixel
    if not (0 <= current_y < h and 0 <= current_x < w) or skeleton[current_y, current_x] == 0:
        return None, []
    
    # Check if this path was already traced globally
    if visited_global[current_y, current_x]:
        return None, []

    # Initialize path and a local visited set
    path = [start_node, (current_y, current_x)]
    visited_on_this_path = set(path)
    queue = deque([(current_y, current_x, [(current_y, current_x)])])

    while queue:
        cy, cx, current_path = queue.popleft()
        
        # Get all valid neighbors
        neighbors = get_neighbors(cy, cx, skeleton)
        prev_pixel = current_path[-2] if len(current_path) > 1 else None
        candidates = [n for n in neighbors if n != prev_pixel]

        if not candidates:
            continue

        for next_pixel in candidates:
            ny, nx = next_pixel
            
            # Check if this pixel is a node or completes a valid loop
            if nodes[ny, nx] == 1 or (ny, nx) == start_node:
                new_path = current_path + [next_pixel]
                if (ny, nx) == start_node and len(new_path) <= 3:
                    continue  # Skip trivial loops
                return (ny, nx), new_path

            # Continue tracing if not visited on this path and not globally marked
            if (next_pixel not in visited_on_this_path and 
                not visited_global[ny, nx]):
                visited_on_this_path.add(next_pixel)
                queue.append((ny, nx, current_path + [next_pixel]))

        # Mark current pixel as visited globally (except start/end nodes)
        if (cy, cx) not in [start_node, (sy + dy, sx + dx)]:
            visited_global[cy, cx] = True

    return None, []

def build_graph_from_skeleton(skeleton, nodes):
    """
    Build Graph from Skeleton and Nodes
    """
    G = networkx.Graph()
    h, w = skeleton.shape
    
    # Get coordinates of all nodes and add them to the graph
    node_coords = list(zip(*np.where(nodes == 1)))
    for y, x in node_coords:
        G.add_node((y, x))
    
    # Use a single, robust tracing pass to find all edges
    visited_pixels = np.zeros_like(skeleton, dtype=bool)
    
    # Mark node locations as "visited" to not trace over them
    visited_pixels[nodes == 1] = True
    
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                  (-1, -1), (-1, 1), (1, -1), (1, 1)]
    
    # For each node, trace paths starting in all 8 directions
    for start_node in node_coords:
        sy, sx = start_node
        
        for dy, dx in directions:
            # Start tracing from the pixel adjacent to the node
            ny, nx = sy + dy, sx + dx
            
            # Skip if out of bounds, not on skeleton, or already part of an edge
            if not (0 <= ny < h and 0 <= nx < w) or skeleton[ny, nx] == 0 or visited_pixels[ny, nx]:
                continue
            
            # Trace the path until we hit the next node
            end_node, path = trace_path_between_nodes(start_node, (dy, dx), skeleton, nodes, visited_pixels)
            
            if end_node is not None and len(path) > 1:
                # Add the edge to the graph
                u, v = tuple(sorted((start_node, end_node)))
                if not G.has_edge(u, v):
                    G.add_edge(u, v, path=path, length=len(path))

                # Mark all pixels in this path as visited (excluding nodes)
                for py, px in path[1:-1]:
                    visited_pixels[py, px] = True
            
    return G

def add_node_features(G):
    """Add node features for graph analysis"""
    for node in G.nodes():
        G.nodes[node]['x'] = np.array([node[0]/100, node[1]/100])  # normalize
    return G


# draw image

def draw_graph_as_image(graph, image_shape, line_thickness=2, node_size=5):
    """
    Draw the graph structure as a binary image.
    
    Args:
        graph: NetworkX graph with path information
        image_shape: Tuple (height, width) of output image
        line_thickness: Thickness of drawn lines
        node_size: Size of drawn nodes
    
    Returns:
        Binary image with graph structure drawn
    """
    # Create blank image
    image = np.zeros(image_shape, dtype=np.uint8)
    
    if graph.number_of_nodes() == 0:
        return image
    
    # Draw edges with their paths
    for u, v, data in graph.edges(data=True):
        if 'path' in data and len(data['path']) > 1:
            # Draw the actual path
            path = np.array(data['path'])
            for i in range(len(path) - 1):
                y1, x1 = int(path[i][0]), int(path[i][1])
                y2, x2 = int(path[i+1][0]), int(path[i+1][1])
                
                # Ensure coordinates are within bounds
                if (0 <= y1 < image_shape[0] and 0 <= x1 < image_shape[1] and
                    0 <= y2 < image_shape[0] and 0 <= x2 < image_shape[1]):
                    cv2.line(image, (x1, y1), (x2, y2), 255, line_thickness)
        else:
            # Draw straight line between nodes
            y1, x1 = int(u[0]), int(u[1])
            y2, x2 = int(v[0]), int(v[1])
            
            if (0 <= y1 < image_shape[0] and 0 <= x1 < image_shape[1] and
                0 <= y2 < image_shape[0] and 0 <= x2 < image_shape[1]):
                cv2.line(image, (x1, y1), (x2, y2), 255, line_thickness)
    
    # Draw nodes
    for node in graph.nodes():
        y, x = int(node[0]), int(node[1])
        if 0 <= y < image_shape[0] and 0 <= x < image_shape[1]:
            cv2.circle(image, (x, y), node_size, 255, -1)
    
    return image

def create_enhanced_graph_image(graph, image_shape, style='clean'):
    """
    Create enhanced visualization of graph structure.
    
    Args:
        graph: NetworkX graph
        image_shape: Tuple (height, width)
        style: 'clean', 'detailed', or 'artistic'
    
    Returns:
        Enhanced graph image
    """
    if style == 'clean':
        return draw_graph_as_image(graph, image_shape, line_thickness=2, node_size=3)
    
    elif style == 'detailed':
        # Create multi-channel image for detailed view
        image = np.zeros((*image_shape, 3), dtype=np.uint8)
        
        # Draw edges in blue
        for u, v, data in graph.edges(data=True):
            if 'path' in data and len(data['path']) > 1:
                path = np.array(data['path'])
                for i in range(len(path) - 1):
                    y1, x1 = int(path[i][0]), int(path[i][1])
                    y2, x2 = int(path[i+1][0]), int(path[i+1][1])
                    
                    if (0 <= y1 < image_shape[0] and 0 <= x1 < image_shape[1] and
                        0 <= y2 < image_shape[0] and 0 <= x2 < image_shape[1]):
                        cv2.line(image, (x1, y1), (x2, y2), (255, 100, 100), 2)
        
        # Draw nodes in different colors based on degree
        for node in graph.nodes():
            y, x = int(node[0]), int(node[1])
            if 0 <= y < image_shape[0] and 0 <= x < image_shape[1]:
                degree = graph.degree(node)
                if degree == 1:  # Endpoint
                    color = (0, 255, 0)  # Green
                    size = 4
                elif degree == 2:  # Regular node
                    color = (255, 255, 0)  # Yellow
                    size = 3
                else:  # Junction
                    color = (0, 0, 255)  # Red
                    size = 6
                
                cv2.circle(image, (x, y), size, color, -1)
        
        return image
    
    elif style == 'artistic':
        # Create artistic representation with varying line thickness
        image = np.zeros(image_shape, dtype=np.uint8)
        
        # Draw edges with thickness based on path length
        for u, v, data in graph.edges(data=True):
            thickness = max(1, min(5, data.get('length', 10) // 5))
            
            if 'path' in data and len(data['path']) > 1:
                path = np.array(data['path'])
                for i in range(len(path) - 1):
                    y1, x1 = int(path[i][0]), int(path[i][1])
                    y2, x2 = int(path[i+1][0]), int(path[i+1][1])
                    
                    if (0 <= y1 < image_shape[0] and 0 <= x1 < image_shape[1] and
                        0 <= y2 < image_shape[0] and 0 <= x2 < image_shape[1]):
                        cv2.line(image, (x1, y1), (x2, y2), 255, thickness)
        
        # Apply slight blur for artistic effect
        image = cv2.GaussianBlur(image, (3, 3), 0)
        
        return image

# gradio ui
def save_uploaded_file_tab6(uploaded_file, save_dir="uploaded_image"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    timestamp = str(int(time.time() * 1000))  # milliseconds
    filename = f"uploaded_{timestamp}.jpeg"
    save_path = os.path.join(save_dir, filename)

    if uploaded_file.mode == 'RGBA':
        uploaded_file = uploaded_file.convert("RGB")
    
    uploaded_file.save(save_path)
    return save_path

def image_processing_tab6(img):
    img_path = save_uploaded_file_tab2(img)
    process_image = process_signature_image(img_path)

    
    binary_signature_cv2 = process_image[0]
    
    # Step 2: Generate skeleton
    skeleton_cv2 = skeletonize_image(binary_signature_cv2)

    vector = extract_vector(binary_signature_cv2)

    vector_memory.store(vector)

    # Step 3: Detect nodes
    nodes_cv2 = detect_nodes(skeleton_cv2)

    # Step 4: Build graph
    graph = build_graph_from_skeleton(skeleton_cv2, nodes_cv2)
    graph = add_node_features(graph)

    # Step 5: Generate graph-based images
    image_shape = binary_signature_cv2.shape

    # For node detection visualization: Overlay nodes on skeleton
    node_detection_viz = skeleton_cv2.copy().astype(float)
    node_coords = np.where(nodes_cv2 == 1)
    if len(node_coords[0]) > 0:
        # Ensure that node_detection_viz is a color image if we want colored nodes
        node_detection_viz = cv2.cvtColor((node_detection_viz * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
        for y, x in zip(node_coords[0], node_coords[1]):
            cv2.circle(node_detection_viz, (x, y), 5, (0, 0, 255), -1) # Red circles for nodes

    # Convert to PIL Image
    binary_signature = Image.fromarray(binary_signature_cv2)
    skeleton_signature = Image.fromarray(skeleton_cv2 * 255) # Scale to 0-255 for display
    node_signature = Image.fromarray(node_detection_viz.astype(np.uint8)) # Cast back to uint8
    
    # Graph structure visualization (drawing lines and nodes)
    graph_structure_image_cv2 = draw_graph_as_image(graph, image_shape, line_thickness=2, node_size=3)
    graph_structure = Image.fromarray(graph_structure_image_cv2)

    # Clean graph image
    clean_graph_image_cv2 = create_enhanced_graph_image(graph, image_shape, style='clean')
    clean_graph = Image.fromarray(clean_graph_image_cv2)

    # Enhanced graph image
    enhanced_graph_image_cv2 = create_enhanced_graph_image(graph, image_shape, style='detailed')
    
    # If the enhanced image is a 3-channel (color) image, ensure it's RGB for PIL
    if len(enhanced_graph_image_cv2.shape) == 3:
        enhanced_graph = Image.fromarray(cv2.cvtColor(enhanced_graph_image_cv2, cv2.COLOR_BGR2RGB))
    else:
        enhanced_graph = Image.fromarray(enhanced_graph_image_cv2)

    return (binary_signature, skeleton_signature, node_signature, graph_structure, clean_graph, enhanced_graph, vector)


def cosine_similarity_tab6():
    vec1, vec2 = vector_memory.get_vectors()

    if vec1 is None or vec2 is None:
        return "Both vectors not available"

    if vec1.shape != vec2.shape:
        return f"Shape mismatch: {vec1.shape} vs {vec2.shape}"

    score = float(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))
    return round(score, 4)



In [ ]:
Tab 6: Global Vector Memory

In [20]:
class VectorMemory:
    def __init__(self):
        self.vectors = [None, None]
        self.index = 0  # 0 for vector1, 1 for vector2

    def store(self, vec):
        self.vectors[self.index] = vec
        self.index = 1 - self.index  # Toggle between 0 and 1

    def get_vectors(self):
        return self.vectors[0], self.vectors[1]

    def reset(self):
        self.vectors = [None, None]
        self.index = 0

vector_memory = VectorMemory()


In [ ]:
# facebook/DINOv2

In [74]:
import torch
from transformers import AutoProcessor, AutoModel
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load DINOv2 model and processor (downloaded from Hugging Face)
processor_dino = AutoProcessor.from_pretrained("facebook/dinov2-base")
model_dino = AutoModel.from_pretrained("facebook/dinov2-base", local_files_only=True)

# Function to extract image features
def extract_vector_dino(image_array):
    image = Image.fromarray(cv2.cvtColor(image_array, cv2.COLOR_GRAY2RGB))
    inputs = processor_dino(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model_dino(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling
    vector = torch.nn.functional.normalize(embeddings, p=2, dim=1)  # Normalize
    # Convert to numpy and squeeze to make it a 1D array
    vector = vector.squeeze().cpu().numpy()
    # The line below `vector /= np.linalg.norm(vector)` is redundant if already normalized by torch.nn.functional.normalize
    # but won't cause harm if you want to ensure it's unit length in numpy as well.
    # vector /= np.linalg.norm(vector)
    return vector

'''
def extract_vector(image_array):
    image = Image.fromarray(cv2.cvtColor(image_array, cv2.COLOR_GRAY2RGB))
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token embedding
    vector = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    vector /= np.linalg.norm(vector)
    return vector
'''
    
def cosine_similarity_dino(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def get_similarity_dino(check_signature, stored_signature):
    vec1 = extract_vector_dino(check_signature)
    vec2 = extract_vector_dino(stored_signature)
    similarity = cosine_similarity_dino(vec1, vec2)
    return similarity



In [75]:
import matplotlib.pyplot as plt

def get_process_signature_dino(check_path, signature_path):
    check_images = process_signature_image(check_path)
    stored_images = process_signature_image(signature_path)

    if not check_images or not stored_images:
        print("No signatures detected.")
        return

    check_signature = check_images[0]
    stored_signature = stored_images[0]

    similarity = get_similarity_dino(check_signature, stored_signature)
    print("Cosine Similarity:", similarity)

    plot_signature(check_signature, stored_signature)

    if similarity >= 0.80:
        print("Genuine Signature")
    else:
        print("Forged Signature")


def plot_signature(check_signature, stored_signature):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(check_signature, cmap='gray')
    axs[0].set_title("Uploaded Signature")
    axs[0].axis('off')

    axs[1].imshow(stored_signature, cmap='gray')
    axs[1].set_title("Reference Signature")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()


In [86]:


get_process_signature_dino("check_signature_test_case/signature/tareq_1.jpeg", "check_signature_test_case/signature/tareq_2.jpeg")



0: 640x576 1 signature, 71.3ms
Speed: 1.8ms preprocess, 71.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 576)

0: 416x640 1 signature, 52.4ms
Speed: 1.2ms preprocess, 52.4ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 640)
Cosine Similarity: 0.7387411


<Figure size 1000x400 with 2 Axes>

Forged Signature
